In [2]:
import h5py
import numpy as np
import pickle

# Make the Dataset from Window
- Let's challenge model with smaller and smaller window

# Load the v.c. 1/3 window hdf5 file

In [3]:
train_X = np.zeros((7856, 200, 200, 1)); train_Y = np.zeros((7856, 2))
val_X = np.zeros((1680, 200, 200, 1)); val_Y = np.zeros((1680, 2))
test_X = np.zeros((1680, 200, 200, 1)); test_Y = np.zeros((1680, 2))

In [4]:
dir = r'C:\Users\bono2\Desktop\Pitching Airfoil\dataset\window_normalized'

with open(dir + r"\index.pickle", 'rb') as file:
    idx = pickle.load(file)

In [5]:
hdf5_dir = "C:/Users/bono2/Desktop/Pitching Airfoil/dataset/window/window_vertical_concat_dataset.hdf5"
file = h5py.File(hdf5_dir, 'r')

for i, index in zip(range(train_X.shape[0]), idx['train']):
    train_X[i] = file['u'][index].reshape(200, 200, 1)
    train_Y[i] = file['label'][index]

for i, index in zip(range(val_X.shape[0]), idx['val']):
    val_X[i] = file['u'][index].reshape(200, 200, 1)
    val_Y[i] = file['label'][index]
    
for i, index in zip(range(test_X.shape[0]), idx['test']):
    test_X[i] = file['u'][index].reshape(200, 200, 1)
    test_Y[i] = file['label'][index]

# Make it 1/5window

In [6]:
train_X = np.concatenate((train_X[:, 20:80, 79:199], train_X[:, 120:180, 79:199]), axis = 1)
val_X = np.concatenate((val_X[:, 20:80, 79:199], val_X[:, 120:180, 79:199]), axis = 1)
test_X = np.concatenate((test_X[:, 20:80, 79:199], test_X[:, 120:180, 79:199]), axis = 1)

## Transformation to Sparse
- Give stride = 3
- Becomes 40 x 40 data

In [7]:
train_X = np.concatenate((train_X[:, 0:60:3, ::3], train_X[:, 61::3, ::3]), axis = 1)
val_X = np.concatenate((val_X[:, 0:60:3, ::3], val_X[:, 61::3, ::3]), axis = 1)
test_X = np.concatenate((test_X[:, 0:60:3, ::3], test_X[:, 61::3, ::3]), axis = 1)

In [8]:
print(train_X.shape, val_X.shape, test_X.shape)

(7856, 40, 40, 1) (1680, 40, 40, 1) (1680, 40, 40, 1)


# Zero Centering

In [8]:
# Get mean for zero centering
mean = np.mean(train_X, axis = 0)
train_X -= mean
val_X -= mean
test_X -= mean

# Min-Max Scaling

In [9]:
train_ux = train_X[:,0:20,:,:]; train_uy = train_X[:,20:,:,:]
print(train_ux.shape, train_uy.shape)

val_ux = val_X[:,0:20,:,:]; val_uy = val_X[:,20:,:,:]
test_ux = test_X[:,0:20,:,:]; test_uy = test_X[:,20:,:,:]

ux_max = 0; ux_min = 10000; uy_max = 0; uy_min = 10000

for ux in train_ux:
    M = np.max(ux); m = np.min(ux)
    if ux_max < M:
        ux_max = M
    elif ux_min > m:
        ux_min = m
   
for uy in train_uy:
    M = np.max(uy); m = np.min(uy)
    if uy_max < M:
        uy_max = M
    if uy_min > m:
        uy_min = m
        
print(ux_max, uy_max, ux_min, uy_min)
ux_range = ux_max - ux_min
uy_range = uy_max - uy_min

train_ux /= ux_range; train_uy /= uy_range
val_ux /= ux_range; val_uy /= uy_range
test_ux /= ux_range; test_uy /= uy_range

(7856, 38, 75, 1) (7856, 37, 75, 1)
0.20410246601323845 0.39137964902058303 -0.2243675300407344 -0.4121790080400961


In [10]:
for x, y, i in zip(train_ux, train_uy, range(len(train_uy))):
    train_X[i,0:20,:,:] = x
    train_X[i,20:,:,:] = y
for x, y, i in zip(val_ux, val_uy, range(len(val_uy))):
    val_X[i,0:20,:,:] = x
    val_X[i,20:,:,:] = y
for x, y, i  in zip(test_ux, test_uy, range(len(val_uy))):
    test_X[i,0:20,:,:] = x
    test_X[i,20:,:,:] = y

# Save the Data

In [11]:
save_dir =  r'C:\Users\bono2\Desktop\Pitching Airfoil\dataset\sparse_sensing_smaller_window\one_fifth_window\sparse_one_fifteenth'

In [12]:
np.save(save_dir + r"\train_X.npy", train_X); np.save(save_dir + r"\train_Y.npy", train_Y)
np.save(save_dir + r"\val_X.npy", val_X); np.save(save_dir + r"\val_Y.npy", val_Y)
np.save(save_dir + r"\test_X.npy", test_X); np.save(save_dir + r"\test_Y.npy", test_Y)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\bono2\\Desktop\\Pitching Airfoil\\dataset\\sparse_sensing_smaller_window\\sparse_one_eighth\\train_X.npy'